# Preparations

In [1]:
import pyterrier as pt
import pyterrier_rag.readers
from pyterrier_rag.backend import HuggingFaceBackend
import pandas as pd

import agent_framework
from agent_framework import R1Searcher
from agent_framework import SearchR1

INFO 08-26 02:01:46 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-26 02:01:46 [__init__.py:239] Automatically detected platform cuda.


In [2]:
ds = pt.get_dataset('rag:nq') # or rag:hotpotqa
TOPICS = ds.get_topics('dev').head(100)
QRELS  = ds.get_answers('dev')

Java started (triggered by _pt_tokeniser) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [7]:
sparse_index = pt.Artifact.from_hf('pyterrier/ragwiki-terrier')
bm25_ret = pt.rewrite.tokenise() >> sparse_index.bm25(include_fields=['docno', 'text', 'title'], threads=5, verbose = True) >> pt.rewrite.reset()

In [15]:
MEAS = [
    pyterrier_rag.measures.F1,
    pyterrier_rag.measures.EM,
    # "mrt",
    # "llm_time",
    # "retrieval_time",
    # "turns",
    # "gpu_usage"
]

# RQ1--LLM calls


In [13]:
# ---- 四个系统变体 ----
hf_7b = R1Searcher(
    retriever=bm25_ret,
    model_id="XXsongLALA/Qwen-2.5-7B-base-RAG-RL",
    use_vllm=False,                 # 强制 HF 顺序生成
    max_turn=6, top_k=8, max_tokens=512, verbose=True, prompt_type='v1'
)

vllm_7b = R1Searcher(
    retriever=bm25_ret,
    model_id="XXsongLALA/Qwen-2.5-7B-base-RAG-RL",
    use_vllm=True,                  # vLLM 并行生成
    max_turn=6, top_k=8, max_tokens=512, verbose=True, prompt_type='v1'
)

vllm_7b_smallGPU = R1Searcher(
    retriever=bm25_ret,
    model_id="XXsongLALA/Qwen-2.5-7B-base-RAG-RL",
    use_vllm=True,
    model_kw_args=dict(             # 更保守的显存/上下文配置
        gpu_memory_utilization=0.60,
        max_model_len=768,
        tensor_parallel_size=1
    ),
    max_turn=6, top_k=8, max_tokens=384, verbose=True, prompt_type='v1'
)

vllm_3b = R1Searcher(
    retriever=bm25_ret,
    model_id="Qwen/Qwen2.5-3B-Instruct",   # 可换成你本地已可用的小模型
    use_vllm=True,
    max_turn=6, top_k=8, max_tokens=384, verbose=True, prompt_type='v1'
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[R1Searcher] transformers backend ready: XXsongLALA/Qwen-2.5-7B-base-RAG-RL on cuda:0
INFO 08-26 02:22:27 [config.py:717] This model supports multiple tasks: {'generate', 'score', 'reward', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 08-26 02:22:27 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 08-26 02:22:27 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 08-26 02:22:28 [utils.py:2382] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
WARNING 08-26 02:22:33 [utils.py:2522] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x7f1

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  5.30it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:00<00:00,  3.68it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.16it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.46it/s]



ERROR 08-26 02:22:37 [core.py:396] EngineCore failed to start.
ERROR 08-26 02:22:37 [core.py:396] Traceback (most recent call last):
ERROR 08-26 02:22:37 [core.py:396]   File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 387, in run_engine_core
ERROR 08-26 02:22:37 [core.py:396]     engine_core = EngineCoreProc(*args, **kwargs)
ERROR 08-26 02:22:37 [core.py:396]   File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 329, in __init__
ERROR 08-26 02:22:37 [core.py:396]     super().__init__(vllm_config, executor_class, log_stats,
ERROR 08-26 02:22:37 [core.py:396]   File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 71, in __init__
ERROR 08-26 02:22:37 [core.py:396]     self._initialize_kv_caches(vllm_config)
ERROR 08-26 02:22:37 [core.py:396]   File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 133, in _initialize_kv_caches
ERROR 08-26 02:22:37 [core.py:396]     kv_cache_

Process EngineCore_0:
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 400, in run_engine_core
    raise e
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 387, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 329, in __init__
    super().__init__(vllm_config, executor_class, log_stats,
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 71, in __init__
    self._initialize_kv_caches(vllm_config)
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 133, in _initialize_kv_caches
    kv_cache

...

Process EngineCore_0:
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 400, in run_engine_core
    raise e
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 387, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 329, in __init__
    super().__init__(vllm_config, executor_class, log_stats,
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 64, in __init__
    self.model_executor = executor_class(vllm_config)
  File "/opt/miniconda3/lib/python3.10/site-packages/vllm/executor/executor_base.py", line 52, in __init__
    self

[R1Searcher] vLLM init failed, fallback to transformers: Engine core initialization failed. See root cause above.


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[R1Searcher] transformers backend ready: Qwen/Qwen2.5-3B-Instruct on cuda:0


In [16]:
rq1_df = pt.Experiment(
    [hf_7b, vllm_7b, vllm_7b_smallGPU, vllm_3b],
    TOPICS, QRELS, MEAS,
    names=[
        "HF-7B (sequential)",
        "vLLM-7B (parallel)",
        "vLLM-7B (small-GPU)",
        "vLLM-3B (parallel)"
    ],
    batch_size=8,
    verbose=True
)
rq1_df

pt.Experiment:   0%|          | 0/52 [00:00<?, ?batches/s]


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_SUPPORTED when calling `cublasLtMatmulAlgoGetHeuristic( ltHandle, computeDesc.descriptor(), Adesc.descriptor(), Bdesc.descriptor(), Cdesc.descriptor(), Cdesc.descriptor(), preference.descriptor(), 1, &heuristicResult, &returnedResult)`